In [10]:
!pip install sqlalchemy
!pip install pymysql
!pip install beautifulsoup4
!pip install requests


In [2]:
import pandas as pd
import sqlalchemy
import pymysql
import requests
from bs4 import BeautifulSoup
import re
import datetime as dt

In [28]:
def connection_with_sql():
    schema = "cities_info"
    host = "host_id"
    user = "root"
    password = "Password"
    port = 3306

    connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'
    return connection_string

In [30]:
def cities_data(NewcityEntry):
    cities_ration=[]
    for i in NewcityEntry:
        url = "https://en.wikipedia.org/wiki/"+i
        response = requests.get(url)
        cities_soup = BeautifulSoup(response.content, 'html.parser')
        city_ration={
            "City" : i,
            "Country" : cities_soup.find(string=re.compile("Country")).find_next("td").get_text(),
            "Latitude" : cities_soup.find(class_="latitude").get_text(),
            "Longitude" : cities_soup.find(class_="longitude").get_text()}
        cities_ration.append(city_ration)
    cities_first_df= pd.DataFrame(cities_ration)
    cities_first_df.to_sql('cities',
                  if_exists='append',
                  con=connection_with_sql(),
                  index=False)
    return pd.DataFrame(cities_ration)

In [32]:
cities_final = cities_data(NewcityEntry=["Munich", "Hamburg", "Berlin"])
cities_final

,City,Country,Latitude,Longitude
0,Munich,Germany,48°08′15″N,11°34′30″E
1,Hamburg,Germany,53°33′N,10°00′E
2,Berlin,Germany,52°31′12″N,13°24′18″E


In [34]:
def cities_population():
    cities_from_sql = pd.read_sql("cities", con=connection_with_sql())
    cities_pop=[]
    for i in cities_from_sql["City"]:
        url = "https://en.wikipedia.org/wiki/"+i
        response = requests.get(url)
        cities_soup = BeautifulSoup(response.content, 'html.parser')
        pop = cities_soup.find(string=("Population")).find_next("td").get_text()
        pop_int = int(pop.replace(",",""))
        city_pop={
            "City" : i,
            "Population" : pop_int,
            "Year_Data_Retrieved" : dt.datetime.now().strftime('%Y-%m-%d')
        }
        cities_pop.append(city_pop)
    cities_pop_df= pd.DataFrame(cities_pop)
    cities_id_pop_df = cities_pop_df.merge(cities_from_sql,
                                           on = "City",
                                           how="left")[["city_id","Population","Year_Data_Retrieved"]]
    cities_id_pop_df.to_sql('population',
                            if_exists='append',
                            con=connection_with_sql(),
                            index=False)
    return 

In [36]:
cities_population()

In [34]:
def cities_weather():
    cities_from_sql = pd.read_sql("cities", con=connection_with_sql())
# Create an empty list which will be filled with the average temp, main weather, description of weather and  time.
    forecasts_data = []
    for i, row in cities_from_sql.iterrows():
        city_name_forec = row["City"]
        API_key = "weather_API"
        weather_5_day_per_city= requests.get(f"https://api.openweathermap.org/data/2.5/forecast?q={city_name_forec}&units=metric&&appid={API_key}")
        weather_5_day_per_city = weather_5_day_per_city.json()

# Create an empty list which will be filled with the average temp, main weather, description of weather and  time.

# Loop through the forecasts
        for forecast in weather_5_day_per_city["list"]:
            forecasts_dict = { 
                "City" : city_name_forec,
                "Avg_temperature"     : forecast["main"]["temp"],
                "Max_temperature"     : forecast["main"]["temp_max"],
                "Min_temperature"     : forecast["main"]["temp_min"],
                "Weather_prediction"  : forecast["weather"][0]["main"],
                "Weather_description" : forecast["weather"][0]["description"],
                "Forecast_time"       : forecast["dt_txt"],
                "Timestamp_retrieved" : dt.datetime.now().strftime("%Y-%m-%d")
            }
            forecasts_data.append(forecasts_dict)
        forecasts_df = pd.DataFrame(forecasts_data)   
        cities_forecast_df = forecasts_df.merge(cities_from_sql,
                                                 on = "City",
                                                 how="left")[["city_id",
                                                         "Avg_temperature",
                                                         "Max_temperature",
                                                         "Min_temperature",
                                                         "Weather_prediction",
                                                         "Weather_description",
                                                         "Forecast_time",
                                                         "Timestamp_retrieved"]]
        cities_forecast_df.to_sql('weather_data_5days',
                             if_exists='append',
                             con=connection_with_sql(),
                             index=False)
    return

In [36]:
cities_weather()

In [38]:
def decimal_coord(coordinate):

    degrees = coordinate[:2]
    decimal_degrees = float(coordinate[3:5])/60
    total_degrees = int(degrees) + float(decimal_degrees)
    direction = 1
    if coordinate[-1] in ("S" or "W"):
        direction = -1
    form_coord = round(total_degrees * direction, 5)
    return form_coord

In [40]:
def cities_airports():
    cities_from_sql = pd.read_sql("cities", con=connection_with_sql())
    airports_list = []
    airports_cities_list = []
    for i, row in cities_from_sql.iterrows():
        
        lat = decimal_coord(row["Latitude"])
        long = decimal_coord(row["Longitude"])

        url = f"https://aerodatabox.p.rapidapi.com/airports/search/location/{lat}/{long}/km/50/16"

        querystring = {"withFlightInfoOnly":"true"}

        headers = {
            "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com",
            "X-RapidAPI-Key": "flight_API"
        }

        response = requests.get(url, headers=headers, params=querystring)
        airports_data = response.json()
        
        
        for data in airports_data["items"]:
            airport_list = {
                "icao" : data["icao"],
                "airport_iata" : data["iata"],
                "Airport_name" : data["name"],
                "Airport_name_short" : data["shortName"],
                "Country_Code" : data["countryCode"],
                "Timezone" : data["timeZone"],
                "Municipality" : data["municipalityName"]
            }
            airports_list.append(airport_list)
            airport_city_list = {
                "airport_iata" : data["iata"],
                "city_id" : row["city_id"]
            }
            airports_cities_list.append(airport_city_list)
    airports_df = pd.DataFrame(airports_list)
    airports_df.drop_duplicates(inplace=True)
    airports_cities_df = pd.DataFrame(airports_cities_list)


    airports_df.to_sql('airports',
                       if_exists='append',
                       con=connection_with_sql(),
                       index=False)
    airports_cities_df.to_sql('cities_airports',
                       if_exists='append',
                       con=connection_with_sql(),
                       index=False)
    return 



In [42]:
cities_airports()

In [37]:
def flights_info():
    airports_from_sql = pd.read_sql("airports", con=connection_with_sql())

    # Get today date
    today = dt.date.today()
    # Get tomorrow date
    tomorrow = today + dt.timedelta(days=1)
    # Get tomorrow date in string type
    tomorrow_str = dt.date.isoformat(tomorrow)
    tomorrow_first_part_begin = f"{tomorrow_str}T00:00"
    tomorrow_first_part_end = f"{tomorrow_str}T11:59"
    tomorrow_second_part_begin = f"{tomorrow_str}T12:00"
    tomorrow_second_part_end = f"{tomorrow_str}T23:59"
    tomorrow_dict = [(tomorrow_first_part_begin,tomorrow_first_part_end),(tomorrow_second_part_begin,tomorrow_second_part_end)]
    flights_data = []
    for tomorrow_part1,tomorrow_part2 in tomorrow_dict:
        
        for i, row in airports_from_sql.iterrows():
            url = f"https://aerodatabox.p.rapidapi.com/flights/airports/iata/{row["airport_iata"]}/{tomorrow_part1}/{tomorrow_part2}"

            querystring = {"withLeg":"true","direction":"Arrival","withCancelled":"false","withCodeshared":"false","withCargo":"false","withPrivate":"false","withLocation":"false"}

            headers = {
                "x-rapidapi-key": "FLIGHTapi",
                "x-rapidapi-host": "aerodatabox.p.rapidapi.com"
            }

            response = requests.get(url, headers=headers, params=querystring)
            flights = response.json()
        
            
            for flight in flights["arrivals"]:

                flight_data = {
                    "Arrival_time" : flight["arrival"]["scheduledTime"]["utc"],
                    "airport_iata" : row["airport_iata"],
                    "Revised_time" : flight["arrival"].get("revisedTime",{"utc":flight["arrival"]["scheduledTime"]["utc"]})["utc"],
                    "Departure_city" : flight["departure"]["airport"]["name"],
                    "Terminal" : flight["number"]
                }
                flights_data.append(flight_data)
            flight_data_df =pd.DataFrame(flights_data)
    flight_data_df.to_sql('flights',
                      if_exists='append',
                      con=connection_with_sql(),
                      index=False)        
    return  


In [39]:
flights_info()